In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.quantum import TensorProduct
from scipy.optimize import linear_sum_assignment

In [ ]:
def indf(j,k):
    return j*Nz*Nbands + k*Nbands

In [ ]:
def flake_geometry(h_symbolic, syms, params):
    #extract system parameters
    Ny, Nz, Nbands = params["Ny"], params["Nz"], params["Nbands"]

    #extract symbolic variables
    kx_sym, ky_sym, kz_sym = syms 
    
    Ly_nn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * ky_sym * (-1.0)), (ky_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Ly_nn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * ky_sym), (ky_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Ly_nnn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * ky_sym * (-1.0)), (ky_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Ly_nnn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * ky_sym), (ky_sym, -sp.pi, sp.pi))/ (2 * sp.pi)

    Lz_nn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * kz_sym * (-1.0)), (kz_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lz_nn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * kz_sym), (kz_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lz_nnn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * kz_sym * (-1.0)), (kz_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lz_nnn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * kz_sym), (kz_sym, -sp.pi, sp.pi))/ (2 * sp.pi)

    Ly_nn_pos = Ly_nn_pos.rewrite(sp.cos).simplify()
    Ly_nn_neg = Ly_nn_neg.rewrite(sp.cos).simplify()
    Ly_nnn_pos = Ly_nnn_pos.rewrite(sp.cos).simplify()
    Ly_nnn_neg = Ly_nnn_neg.rewrite(sp.cos).simplify()
    Lz_nn_pos = Lz_nn_pos.rewrite(sp.cos).simplify()
    Lz_nn_neg = Lz_nn_neg.rewrite(sp.cos).simplify()
    Lz_nnn_pos = Lz_nnn_pos.rewrite(sp.cos).simplify()
    Lz_nnn_neg = Lz_nnn_neg.rewrite(sp.cos).simplify()

    H_diag = h_symbolic
    H_diag -= (Ly_nn_pos * sp.exp(sp.I * ky_sym) + Ly_nn_neg * sp.exp(-sp.I * ky_sym) + Ly_nnn_pos * sp.exp(sp.I * 2 * ky_sym) + Ly_nnn_neg * sp.exp(-sp.I * 2 * ky_sym))
    H_diag -= (Lz_nn_pos * sp.exp(sp.I * kz_sym) + Lz_nn_neg * sp.exp(-sp.I * kz_sym) + Lz_nnn_pos * sp.exp(sp.I * 2 * kz_sym) + Lz_nnn_neg * sp.exp(-sp.I * 2 * kz_sym))
    H_diag = H_diag.rewrite(sp.cos).simplify()
    H_diag = sp.nsimplify(H_diag, tolerance = 1e-8)

    #create container for the Hamiltonian
    h = sp.zeros(Ny*Nz*Nbands, Ny*Nz*Nbands)

    for j in range(Ny):
        for k in range(Nz):
            h[indf(j,k):indf(j,k+1),indf(j,k):indf(j,k+1)] = H_diag

            if j > 0:
                h[indf(j-1,k):indf(j-1,k+1), indf(j,k):indf(j,k+1)] = Ly_nn_pos

            if j > 1:
                h[indf(j-2,k):indf(j-2,k+1), indf(j,k):indf(j,k+1)] = Ly_nnn_pos

            if j < Ny - 1:
                h[indf(j+1,k):indf(j+1,k+1), indf(j,k):indf(j,k+1)] = Ly_nn_neg

            if j < Ny - 2:
                h[indf(j+2,k):indf(j+2,k+1), indf(j,k):indf(j,k+1)] = Ly_nnn_neg

            if k > 0:
                h[indf(j,k-1):indf(j,k), indf(j,k):indf(j,k+1)] = Lz_nn_pos

            if k > 1:
                h[indf(j,k-2):indf(j,k-1), indf(j,k):indf(j,k+1)] = Lz_nnn_pos

            if k < Nz - 1:
                h[indf(j,k+1):indf(j,k+2), indf(j,k):indf(j,k+1)] = Lz_nn_neg

            if k < Nz - 2:
                h[indf(j,k+2):indf(j,k+3), indf(j,k):indf(j,k+1)] = Lz_nnn_neg

    flake_h = sp.lambdify(kx_sym, h, "numpy")

    return flake_h

In [ ]:
#create the Pauli matrices
s0 = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])

In [ ]:
#create parameters and the Pauli matrices 
kx_sym, ky_sym, kz_sym = sp.symbols('k_x k_y k_z', real = True)
ksymbols = [kx_sym, ky_sym, kz_sym]
alpha_sym = sp.symbols('alpha', real = True, positive = True)
gamma_z, lambda_z = sp.symbols('gamma_z lambda_z', real = True)

In [ ]:
H_dartboard = - sp.sin(kx_sym) * sp.sin(2*ky_sym) * sx
H_dartboard += sp.sin(2*kx_sym) * sp.sin(ky_sym) * sy 
H_dartboard += (alpha_sym + sp.cos(2*kx_sym) + sp.cos(2*ky_sym)) * sz

In [ ]:
H_layered = sp.Matrix(np.zeros((4,4)))
H_layered += TensorProduct(sz,H_dartboard)
H_layered += TensorProduct(sy,s0) * lambda_z * sp.sin(kz_sym)
H_layered += TensorProduct(sx,s0) * (gamma_z + lambda_z * sp.cos(kz_sym))

In [ ]:
H_layered

In [ ]:
#define system size
Nx = 30
Ny = 20
Nz = 20
Nbands = 4
Nocc = 2

params = {}
params["Nx"] = Nx
params["Ny"] = Ny
params["Nz"] = Nz
params["Nbands"] = Nbands
params["Nocc"] = Nocc

In [ ]:
#fix the parameters of the model
H_fixparam = H_layered.subs({alpha_sym : 1.0, gamma_z: 0.5, lambda_z: 1})

In [ ]:
h_flake = flake_geometry(H_fixparam, [kx_sym, ky_sym, kz_sym], params)

In [ ]:
Kxs = np.linspace(0, 2*np.pi, Nx, endpoint = False)

In [ ]:
eigenvalues = np.zeros((Nx, Ny*Nz*Nbands))
eigenstates = np.zeros((Nx,Ny*Nz*Nbands,Ny*Nz*Nbands), dtype = np.complex128)

for i in range(Nx): 
    vals, vecs = np.linalg.eigh(h_flake(Kxs[i]))

    ind = np.argsort(vals)
    eigenvalues[i] = vals[ind]
    eigenstates[i] = vecs[:,ind]

In [ ]:
#carry out linear sum assignment
for i in range(Nx-1):
    v0 = eigenstates[i,:,:]
    v1 = eigenstates[i+1,:,:]
    vals1 = eigenvalues[i+1,:]

    Q = abs(v0.conj().T @ v1)
    ind = linear_sum_assignment(-Q)[1]

    eigenvalues[i+1,:] = vals1[ind]
    eigenstates[i+1,:,:] = v1[:, ind]

In [ ]:
fig = plt.figure(figsize=(4,3))

for i in range(Ny*Nz*Nbands):
    plt.plot(Kxs, eigenvalues[:,i], color = 'black', alpha = 0.05)

plt.xlabel(r"$k_x$", fontsize = 16, labelpad = -3)
plt.ylabel(r"$E$", fontsize = 16, labelpad = -7) 
plt.xticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], [r"$0$", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize = 14)
plt.yticks([-3, 0, 3], fontsize = 14)
plt.show()

In [ ]:
eigenvalues[6,800+4]

In [ ]:
slice = 7
density = np.zeros((Ny,Nz))

for m in range(Ny*Nz*2-4,Ny*Nz*2+4):
    v1 = eigenstates[slice,:,m]
    for j in range(Ny):
        for k in range(Nz):
            density[j,k] += np.sum(abs(v1[indf(j,k):indf(j,k+1)])**2)

In [ ]:
np.sum(density)